<a href="https://colab.research.google.com/github/taegeonyu/HDS-5230-07/blob/main/Week11/Week11_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install R essentials
!apt-get install -y libcurl4-openssl-dev libssl-dev libxml2-dev
!apt-get install -y r-base

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcurl4-openssl-dev is already the newest version (7.81.0-1ubuntu1.20).
libssl-dev is already the newest version (3.0.2-0ubuntu1.19).
libxml2-dev is already the newest version (2.9.13+dfsg-1ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
r-base is already the newest version (4.5.0-2.2204.0).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [2]:
# Install required R libraries
!Rscript -e 'install.packages(c("mlbench", "purrr"))'

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/mlbench_2.1-6.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/purrr_1.0.4.tar.gz'
* installing *source* package ‘mlbench’ ...
** this is package ‘mlbench’ version ‘2.1-6’
** package ‘mlbench’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
using C compiler: ‘gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0’
gcc -I"/usr/share/R/include" -DNDEBUG       -fpic  -g -O2 -ffile-prefix-map=/build/r-base-TYDrW1/r-base-4.5.0=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c init.c -o init.o
gcc -I"/usr/share/R/include" -DNDEBUG       -fpic  -g -O2 -ffile-prefix-map=/build/r-base-TYDrW1/r-base-4.5.0=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2  -c waveform.c -o waveform.o
gcc -shared -L/usr/lib/R/lib -Wl,-Bsymbolic-functions -flto=auto -ffat-

In [3]:
# install rpy2
!pip install -q rpy2

In [4]:
# load rpy2 extension
%load_ext rpy2.ipython

In [5]:
# run R code to generate data
%%R

library(mlbench)
library(purrr)

data("PimaIndiansDiabetes2")
ds <- as.data.frame(na.omit(PimaIndiansDiabetes2))

## Fit logistic regression
logmodel <- glm(diabetes ~ ., data = ds, family = "binomial")

cfs <- coefficients(logmodel)
prednames <- variable.names(ds)[-9]

sz <- 1000000  # WARNING: sz=100 million might crash Colab! Try 1 million first

dfdata <- map_dfc(prednames,
                  function(nm){
                    eval(parse(text = paste0("sample(ds$", nm,
                                             ", size = sz, replace = T)")))
                  })

names(dfdata) <- prednames

pvec <- map((1:8),
            function(pnum){
              cfs[pnum+1] * eval(parse(text = paste0("dfdata$", prednames[pnum])))
            }) %>%
  reduce(`+`) + cfs[1]

dfdata$outcome <- ifelse(1/(1 + exp(-pvec)) > 0.5, 1, 0)

write.csv(dfdata, file = "dfdata.csv", row.names = FALSE)

New names:
• `` -> `...1`
• `` -> `...2`
• `` -> `...3`
• `` -> `...4`
• `` -> `...5`
• `` -> `...6`
• `` -> `...7`
• `` -> `...8`


## XGBoost in Python via scikit-learn and 5-fold CV

In [16]:
# import necessary libraries for the task
import pandas as pd
import time
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [7]:
# read data
df = pd.read_csv('/content/dfdata.csv')
df.shape

(1000000, 9)

In [8]:
df.head()

,pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,outcome
0,3,169,56,24,846,43.6,0.460,32,1
1,0,104,88,41,142,30.4,0.586,25,0
2,1,96,82,33,145,33.3,0.674,50,0
3,0,124,60,13,86,32.0,0.335,37,0
4,2,121,84,37,72,30.1,0.444,32,0


In [9]:
# function for evaluation
def xgboost_eval(df, size):
      # empty dictionary
      results = {}
      # split X and y
      data = df.sample(n = size, replace = True, random_state = 42)
      X = data.drop('outcome', axis = 1).values
      y = data['outcome'].values

      # stratified kfold for cross validation
      cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

      # define model
      model = XGBClassifier(eval_metric = 'logloss')

      # start time
      start = time.time()

      # accuracy score
      scores = cross_val_score(model, X, y, cv = cv, scoring = 'accuracy', n_jobs = -1)
      score = scores.mean()

      # time taken
      time_taken = time.time() - start

      results[size] = {
            'accuracy': score,
            'time_taken': time_taken
        }

      return results

In [10]:
# xgboost with 100 samples
xgboost_eval(df, 100)

{100: {'accuracy': np.float64(0.9200000000000002),
  'time_taken': 1.4070422649383545}}

In [11]:
# xgboost with 1000 samples
xgboost_eval(df, 1000)

{1000: {'accuracy': np.float64(0.9510000000000002),
  'time_taken': 0.7660882472991943}}

In [12]:
# xgboost with 10000 samples
xgboost_eval(df, 10000)

{10000: {'accuracy': np.float64(0.9749000000000001),
  'time_taken': 0.16933250427246094}}

In [13]:
# xgboost with 100000 samples
xgboost_eval(df, 100000)

{100000: {'accuracy': np.float64(0.9879), 'time_taken': 0.9474430084228516}}

In [14]:
# xgboost with 1000000 samples
xgboost_eval(df, 1000000)

{1000000: {'accuracy': np.float64(0.994014), 'time_taken': 7.958971738815308}}

In [15]:
# xgboost with 10000000 samples
xgboost_eval(df, 10000000)

{10000000: {'accuracy': np.float64(0.9953551),
  'time_taken': 83.37399387359619}}